# Voice Mapping

Algorighm for mapping the finite number of voices based on the incoming MIDI messages.

## Requirements

Steal note off:
1. Last IDLE voice.
2. BUSY voice with highest sample count.

Steal note on:
1. BUSY note with same note ID.
2. Last IDLE voice.
3. BUSY voice with highest sample count.

In [1]:
import doctest
import collections

Voice = collections.namedtuple("Voice", [
    "state",
    "note",
    "sample_count",
])

In [21]:
IDLE = 0
BUSY = 1

def voice_map(voices, note, steal):
    """Choose a voice for an incoming note.

    Args:
        voices (list): List of voice tuples to choose from.
        note (int): MIDI ID of incoming note.
        steal (bool): True if note-stealing is enabled - false otherwise.

    Returns:
        int: Index (into `voices`) of selected voice.
        bool: True if the note is being stolen and False otherwise.
    
    Doctest:
    
    ==============
    Steal note off
    ==============
    
    >>> voice_map(
    ...     [(IDLE, 0, 0), (IDLE, 0, 0), (IDLE, 0, 0), (IDLE, 0, 0)],
    ...     1,
    ...     steal=False,
    ... )
    (3, False)
    
    >>> voice_map(
    ...     [(BUSY, 3, 0), (IDLE, 0, 0), (IDLE, 0, 0), (IDLE, 0, 0)],
    ...     1,
    ...     steal=False,
    ... )
    (3, False)
    
    >>> voice_map(
    ...     [(IDLE, 0, 0), (IDLE, 0, 0), (IDLE, 0, 0), (BUSY, 3, 10)],
    ...     1,
    ...     steal=False,
    ... )
    (2, False)
    
    >>> voice_map(
    ...     [(BUSY, 30, 10), (BUSY, 40, 11), (BUSY, 50, 14), (BUSY, 60, 13)],
    ...     30,
    ...     steal=False,
    ... )
    (2, False)
    
    =============
    Steal note on
    =============
    
    >>> voice_map(
    ...     [(IDLE, 0, 0), (BUSY, 30, 0), (IDLE, 0, 0), (BUSY, 3, 10)],
    ...     30,
    ...     steal=True,
    ... )
    (1, True)
    
    >>> voice_map(
    ...     [(BUSY, 30, 10), (BUSY, 40, 11), (BUSY, 50, 14), (BUSY, 60, 13)],
    ...     30,
    ...     steal=True,
    ... )
    (0, True)
    
    >>> voice_map(
    ...     [(BUSY, 30, 10), (BUSY, 40, 11), (IDLE, 50, 0), (BUSY, 60, 13)],
    ...     30,
    ...     steal=True,
    ... )
    (0, True)
    
    >>> voice_map(
    ...     [(BUSY, 31, 10), (BUSY, 40, 11), (IDLE, 50, 0), (BUSY, 60, 13)],
    ...     30,
    ...     steal=True,
    ... )
    (2, False)
    
    >>> voice_map(
    ...     [(BUSY, 31, 10), (BUSY, 40, 11), (BUSY, 50, 15), (BUSY, 60, 13)],
    ...     30,
    ...     steal=True,
    ... )
    (2, False)
    
    """
    
    voices = [Voice(*v) for v in voices]
    
    n_idle = -1
    n_steal = -1
    n_oldest = -1
    
    for n, v in enumerate(voices):        
        if v.state == IDLE:
            n_idle = n
        
        if v.state == BUSY:
            if v.note == note:
                n_steal = n
        
            if n_oldest < 0:
                n_oldest = n
            elif v.sample_count > voices[n_oldest].sample_count:
                n_oldest = n

    if steal and n_steal >= 0:
        return (n_steal, True)
    elif n_idle >= 0:
        return (n_idle, False)
    else:
        return (n_oldest, False)


In [22]:
doctest.testmod(verbose=False)

TestResults(failed=0, attempted=9)